<a href="https://colab.research.google.com/github/ccwu0918/book-sqlfifty/blob/main/ch03-select-from/ch03-select-from.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SQL 的五十道練習：初學者友善的資料庫入門

> 從資料表選擇

In [ ]:
!git clone https://github.com/ccwu0918/book-sqlfifty

Cloning into 'book-sqlfifty'...
remote: Enumerating objects: 186, done.
remote: Counting objects: 100% (186/186), done.
remote: Compressing objects: 100% (122/122), done.
remote: Total 186 (delta 54), reused 173 (delta 44), pack-reused 0
Receiving objects: 100% (186/186), 28.90 MiB | 13.73 MiB/s, done.
Resolving deltas: 100% (54/54), done.


In [ ]:
%cd book-sqlfifty

/content/book-sqlfifty


讀者如果是資料科學的初學者，可以略過下述的程式碼；讀者如果不是資料科學的初學者，欲使用 JupyterLab 執行本章節內容，必須先執行下述程式碼載入所需模組與連接資料庫。

In [ ]:
import sqlite3
import unittest
import json
import os
import numpy as np
import pandas as pd
conn = sqlite3.connect('./databases/imdb.db')
conn.execute("""ATTACH './databases/covid19.db' AS covid19""")
conn.execute("""ATTACH './databases/twElection2020.db' AS twElection2020""")
conn.execute("""ATTACH './databases/nba.db' AS nba""")
conn.execute("""ATTACH './databases/northwind.db' AS Northwind""")
conn.execute("""ATTACH './databases/Chinook_Sqlite.sqlite' AS Chinook""")

In [ ]:
# %%capture
# load the SQL magic extension
# https://github.com/catherinedevlin/ipython-sql
# this extension allows us to connect to DBs and issue SQL command
%load_ext sql

# now we can use the magic extension to connect to our SQLite DB
# use %sql to write an inline SQL command
# use %%sql to write SQL commands in a cell
%sql sqlite:///databases/imdb.db

'Connected: @databases/imdb.db'

In [ ]:
# %LOAD sqlite3 db=../databases/imdb.db timeout=2 shared_cache=true

In [ ]:
%%sql
ATTACH "./databases/covid19.db" AS covid19;
ATTACH "./databases/twElection2020.db" AS twElection2020;
ATTACH "./databases/nba.db" AS nba;
ATTACH "./databases/northwind.db" AS Northwind;
ATTACH "./databases/Chinook_Sqlite.sqlite" AS Chinook;

 * sqlite:///databases/imdb.db
Done.
Done.
Done.


[]

## 複習一下

在第二章「建立學習環境」我們寫作了 SQL 敘述完成哈囉世界、檢視學習資料庫中第一個資料表（依照英文字母順序排列）的「前五列、所有欄」，藉此確認學習環境能夠妥善運行。

In [ ]:
%%sql
SELECT 'Hello, World!';

 * sqlite:///databases/imdb.db
Done.


[('Hello, World!',)]

In [ ]:
%%sql
SELECT *
  FROM actors
 LIMIT 5;

 * sqlite:///databases/imdb.db
Done.


[(1, 'Aamir Khan'),
 (2, 'Aaron Eckhart'),
 (3, 'Aaron Lazar'),
 (4, 'Abbas-Ali Roomandi'),
 (5, 'Abbey Lee')]

在第二章「建立學習環境」我們寫作了 SQL 敘述查詢四個學習資料庫每一個資料表的元資料（Metadata）來獲得每一欄的資訊。

In [ ]:
%%sql
SELECT *
  FROM PRAGMA_TABLE_INFO('actors');

 * sqlite:///databases/imdb.db
Done.


[(0, 'id', 'INTEGER', 0, None, 1), (1, 'name', 'TEXT', 0, None, 0)]

## SQL 敘述的組成

藉由觀察這三個 SQL 敘述，我們可以將 SQL 敘述歸納為以下幾個部分的組成：

- 保留字：具有特定功能的指令，例如 `SELECT`、`FROM` 與 `LIMIT`。
- 符號：具有特定功能的符號，例如 `*` 與 `;`。
- 常數：由使用者給予的資料，例如 `'Hello, World!'`。
- 函數：具有特定邏輯的輸入與輸出對應，例如 `PRAGMA_TABLE_INFO()`。

其中 `SELECT` 是「選擇」欄的保留字，`FROM` 是指定「從」哪個資料表查詢，`LIMIT m` 是讓查詢結果顯示前 `m` 列，`*` 表示「所有欄」，`;` 表示一段 SQL 敘述的結束。我們習慣以 `(m, n)` 來描述一個具有 `m` 列、`n` 欄的資料表或者查詢結果，其中 `m` 不包含欄名那一列，舉例來說 `SELECT 'Hello, World!';` 的查詢結果是 `(1, 1)`。

In [ ]:
%%sql
SELECT 'Hello, World!';

 * sqlite:///databases/imdb.db
Done.


[('Hello, World!',)]

`SELECT * FROM actors LIMIT 5;` 的查詢結果是 `(5, 2)`

In [ ]:
%%sql
SELECT *
  FROM actors
 LIMIT 5;

 * sqlite:///databases/imdb.db
Done.


[(1, 'Aamir Khan'),
 (2, 'Aaron Eckhart'),
 (3, 'Aaron Lazar'),
 (4, 'Abbas-Ali Roomandi'),
 (5, 'Abbey Lee')]

## 查詢結果顯示常數：`SELECT constants`

使用單獨存在的 `SELECT` 保留字指定希望在查詢結果中顯示的常數，常用的常數類別有四種，分別是整數、浮點數、文字與空值，我們可以使用 `TYPEOF()` 函數顯示常數或者資料表欄位的類別，當 `SELECT` 之後有不只一個資料的時候就用逗號 `,` 分隔。

|常數類別|範例|
|:------|:---|
|整數 `integer`|`7`, `19`, `5566`, ...etc.|
|浮點數 `real`|`2.718`, `3.14159`, ...etc.|
|文字 `text`|`'Hello, World!'`, `'SQL'`, ...etc.|
|空值 `null`|`NULL`|

In [ ]:
%%sql
SELECT 5566,
       TYPEOF(5566);

 * sqlite:///databases/imdb.db
Done.


[(5566, 'integer')]

In [ ]:
%%sql
SELECT 2.718,
       TYPEOF(2.718);

 * sqlite:///databases/imdb.db
Done.


[(2.718, 'real')]

In [ ]:
%%sql
SELECT 'Hello, World!',
       TYPEOF('Hello, World!');

 * sqlite:///databases/imdb.db
Done.


[('Hello, World!', 'text')]

In [ ]:
%%sql
SELECT NULL,
       TYPEOF(NULL);

 * sqlite:///databases/imdb.db
Done.


[(None, 'null')]

## 在敘述中添加註解

寫作 SQL 敘述會有需要在其中添加註解（Comments）的時候，註解是不會被關聯式資料庫管理系統執行的說明文字，通常作為與工作上的同事、或者未來的自己（看不懂自己從前所寫的程式是相當常見的），說明 SQL 敘述中值得注意的事項或者邏輯，常用的註解形式有單行註解、行末註解與多行註解。

單行註解：用兩個減號 `--` 標註並且單獨存在一行。

```sql
-- single line comment
SELECT columns
  FROM table
 LIMIT m;
```

行末註解：用兩個減號 `--` 標註，但是置放於一行 SQL 敘述的句尾。

```sql
SELECT columns -- end of line comment
  FROM table   -- end of line comment
 LIMIT m;      -- end of line comment
```

多行註解：用 `/*` 開頭、`*/` 結尾來標註。

```sql
/*
multiple-line comments...
multiple-line comments...
multiple-line comments...
*/
SELECT columns
  FROM table
 LIMIT m;
```

## 為查詢結果限制顯示列數：`LIMIT`

```sql
SELECT columns
  FROM table
 LIMIT m;
```

因為資料表中的觀測值列數可能都有很多筆，為了在有限的版面顯示，我們可以透過 `LIMIT` 保留字讓查詢結果顯示前 `m` 列即可。

In [ ]:
%%sql
SELECT *
  FROM movies
 LIMIT 1;

 * sqlite:///databases/imdb.db
Done.


[(1, 'The Shawshank Redemption', 1994, 9.3, 'Frank Darabont', 142)]

In [ ]:
%%sql
SELECT *
  FROM movies
 LIMIT 3;

 * sqlite:///databases/imdb.db
Done.


[(1, 'The Shawshank Redemption', 1994, 9.3, 'Frank Darabont', 142),
 (2, 'The Godfather', 1972, 9.2, 'Francis Ford Coppola', 175),
 (3, 'The Dark Knight', 2008, 9.0, 'Christopher Nolan', 152)]

## 選擇資料表欄位：`SELECT columns FROM table;`

從資料表選擇欄位的時候使用 `SELECT` 與 `FROM` 保留字分別指定欄位名稱與資料表名稱，若希望從資料表選擇「所有」欄位，可以使用星號（`*`）達成。

In [ ]:
%%sql
SELECT *
  FROM movies
 LIMIT 5;

 * sqlite:///databases/imdb.db
Done.


[(1, 'The Shawshank Redemption', 1994, 9.3, 'Frank Darabont', 142),
 (2, 'The Godfather', 1972, 9.2, 'Francis Ford Coppola', 175),
 (3, 'The Dark Knight', 2008, 9.0, 'Christopher Nolan', 152),
 (4, 'The Godfather Part II', 1974, 9.0, 'Francis Ford Coppola', 202),
 (5, '12 Angry Men', 1957, 9.0, 'Sidney Lumet', 96)]

在 `SELECT` 後加入欄的名稱讓查詢結果只顯示資料表中指定的欄。

In [ ]:
%%sql
SELECT title
  FROM movies
 LIMIT 5;

 * sqlite:///databases/imdb.db
Done.


[('The Shawshank Redemption',),
 ('The Godfather',),
 ('The Dark Knight',),
 ('The Godfather Part II',),
 ('12 Angry Men',)]

在 `SELECT` 後加入欄的名稱讓查詢結果只顯示資料表中指定的多個欄，在不同欄名稱之間用逗號分隔。

In [ ]:
%%sql
SELECT title,
    release_year,
    director
FROM movies
LIMIT 5;

 * sqlite:///databases/imdb.db
Done.


[('The Shawshank Redemption', 1994, 'Frank Darabont'),
 ('The Godfather', 1972, 'Francis Ford Coppola'),
 ('The Dark Knight', 2008, 'Christopher Nolan'),
 ('The Godfather Part II', 1974, 'Francis Ford Coppola'),
 ('12 Angry Men', 1957, 'Sidney Lumet')]

## 為查詢結果取別名：`AS alias`

```sql
SELECT constants AS alias;

SELECT columns AS alias
  FROM table;
```

我們可以透過 `AS` 保留字來為查詢的結果取別名，不論是常數或者是資料表的欄，都能在查詢結果中以指定的名稱顯示。

In [ ]:
%%sql
SELECT 'Hello, World!' AS hello_world,
       TYPEOF('Hello, World!') AS typeof_hello_world;

 * sqlite:///databases/imdb.db
Done.


[('Hello, World!', 'text')]

In [ ]:
%%sql
SELECT title AS movie,
    release_year AS released_in,
    director AS directed_by
FROM movies
LIMIT 5;

 * sqlite:///databases/imdb.db
Done.


[('The Shawshank Redemption', 1994, 'Frank Darabont'),
 ('The Godfather', 1972, 'Francis Ford Coppola'),
 ('The Dark Knight', 2008, 'Christopher Nolan'),
 ('The Godfather Part II', 1974, 'Francis Ford Coppola'),
 ('12 Angry Men', 1957, 'Sidney Lumet')]

## 為查詢結果剔除重複值：`DISTINCT`

```sql
SELECT DISTINCT columns
  FROM table;
```

我們可以透過 `DISTINCT` 保留字來為查詢的結果剔除重複值，舉例來說，在 `imdb` 資料庫的 `movies` 資料表中 `director` 欄的前 10 列可以看到重複出現的導演如 Francis Ford Coppola（執導教父三部曲）與 Peter Jackson（執導魔戒三部曲）。

In [ ]:
%%sql
SELECT director
  FROM movies
 LIMIT 10;

 * sqlite:///databases/imdb.db
Done.


[('Frank Darabont',),
 ('Francis Ford Coppola',),
 ('Christopher Nolan',),
 ('Francis Ford Coppola',),
 ('Sidney Lumet',),
 ('Steven Spielberg',),
 ('Peter Jackson',),
 ('Quentin Tarantino',),
 ('Peter Jackson',),
 ('Sergio Leone',)]

在加入 `DISTINCT` 保留字之後同樣顯示前 10 列，可以清楚發現已經沒有重複出現的導演。

In [ ]:
%%sql
SELECT DISTINCT director
  FROM movies
 LIMIT 10;

 * sqlite:///databases/imdb.db
Done.


[('Frank Darabont',),
 ('Francis Ford Coppola',),
 ('Christopher Nolan',),
 ('Sidney Lumet',),
 ('Steven Spielberg',),
 ('Peter Jackson',),
 ('Quentin Tarantino',),
 ('Sergio Leone',),
 ('Robert Zemeckis',),
 ('David Fincher',)]

## SQL 風格指南

閱讀到這裡，讀者對於寫作過的 SQL 敘述應該有一些疑惑，例如保留字大寫、換行或者句首的空白（縮排，Indentation），如果沒有遵照這樣的方式寫作會不會影響查詢結果呢？答案是「不會」，SQL 具有幾個語言特性：

1. 一段 SQL 敘述最後要以分號 `;` 做為結束的標註。
2. 保留字的大小寫並不會影響執行結果，也就是俗稱的大小寫不敏感（Case insensitive）。
3. 常數有固定的書寫方式，像是文字要用一組單引號 `'some texts'` 標註，整數、浮點數與空值可以直接寫作。
4. 保留字之間要以空格或者換行來區隔。

在 SQL 的敘述中保留字大小寫、是否換行或者是否有縮排，都不會對查詢結果有任何的影響。例如下列的這段 SQL 敘述，保留字大小寫、換行與縮排都相當隨興，但卻沒有影響到查詢的結果。

In [ ]:
%%sql
Select title, release_year,
       rating
from movies liMiT 5;

 * sqlite:///databases/imdb.db
Done.


[('The Shawshank Redemption', 1994, 9.3),
 ('The Godfather', 1972, 9.2),
 ('The Dark Knight', 2008, 9.0),
 ('The Godfather Part II', 1974, 9.0),
 ('12 Angry Men', 1957, 9.0)]

唯一不能夠隨興寫作的是 SQL 敘述中保留字彼此之間的「相對順序」，例如目前所學的幾個保留字 `SELECT`、`FROM`、`LIMIT` 等。

```sql
SELECT DISTINCT columns AS alias
  FROM table
 LIMIT m;
```

如果調動保留字的相對順序，就會得到語法錯誤的訊息（Syntax error），例如對調 `LIMIT` 與 `FROM` 的順序。

```sql
SELECT DISTINCT director AS distinct_director
 LIMIT 10
  FROM movies;
```

```
near "FROM": syntax error while preparing "SELECT DISTINCT director AS distinct_director
 LIMIT 10
  FROM movies;".
```

又或者對調 `SELECT` 與 `FROM` 的順序。

```sql
  FROM movies
SELECT DISTINCT director AS distinct_director
 LIMIT 10;
```

```
near "FROM": syntax error while preparing "  FROM movies
SELECT DISTINCT director AS distinct_director
 LIMIT 10;".
```

只需要遵守保留字的書寫順序讓 SQL 寫作時有很大的彈性，但是這不代表只要查詢結果是正確的，就可以隨心所欲地寫作。有時候我們必須顧慮到可讀性（Readibility），特別是在工作時，可讀性更為重要，因為工作上可能會遭遇到協作、程式碼審查（Code review）、維運、代理或者交接等情境，在這些時候查詢結果正確僅是最低門檻，可讀性必須要能達到前述情境中所要求的程度。推薦的作法是參考一份工作團隊、協作夥伴或者自己喜歡並且願意去遵從的風格指南（Style guide），風格指南指的是一套規範程式語言在寫作時必須遵從的編排、格式和設計的準則，風格指南能夠確保每一段程式碼都和其他不同人寫作的程式碼有高度一致性，被眾多使用者採用的 SQL 風格指南有：

- Simon Holywell 的風格指南 <https://www.sqlstyle.guide>
- GitLab 的風格指南 <https://about.gitlab.com/handbook/business-technology/data-team/platform/sql-style-guide>
- Mozilla 的風格指南 <https://docs.telemetry.mozilla.org/concepts/sql_style.html>

本書採用 Simon Holywell 的風格指南來寫作 SQL，其中值得注意的幾個重點有：

- 敘述換行與縮排是採用靠右對齊的編排格式。
- 保留字與函數採用全大寫。
- 取別名的時候採用蛇形命名法（Snake case），例如 `alias_for_some_variables`。

In [ ]:
%%sql
SELECT DISTINCT director AS distinct_director
  FROM movies
 LIMIT 10;

 * sqlite:///databases/imdb.db
Done.


[('Frank Darabont',),
 ('Francis Ford Coppola',),
 ('Christopher Nolan',),
 ('Sidney Lumet',),
 ('Steven Spielberg',),
 ('Peter Jackson',),
 ('Quentin Tarantino',),
 ('Sergio Leone',),
 ('Robert Zemeckis',),
 ('David Fincher',)]

除了參閱、遵從 Simon Holywell 的風格指南，我們也可以善用 SQLiteStudio 的 Format SQL 功能，在編輯器範圍按右鍵，讓寫作的 SQL 敘述之編排、格式和設計具備更高的可讀性。

![](https://github.com/ccwu0918/book-sqlfifty/blob/main/images/format-01.png?raw=1)

![](https://github.com/ccwu0918/book-sqlfifty/blob/main/images/format-02.png?raw=1)

## 重點統整

- 我們可以將 SQL 敘述歸納為以下幾個部分的組成：
    - 保留字：具有特定功能的指令。
    - 符號：具有特定功能的符號。
    - 常數：由使用者給予的資料。
    - 函數：具有特定邏輯的輸入與輸出對應。
- 本書採用 Simon Holywell 的風格指南來寫作 SQL，其中值得注意的幾個重點有：
    - 敘述換行與縮排是採用靠右對齊的編排格式。
    - 保留字與函數採用全大寫。
    - 取別名的時候採用蛇形命名法（Snake case）。
- 這個章節學起來的 SQL 保留字：
    - `SELECT`
    - `FROM`
    - `LIMIT`
    - `AS`
    - `DISTINCT`
- 將截至目前所學的 SQL 保留字集中在一個敘述中，寫作順序必須遵從標準 SQL 的規定。

```sql
SELECT DISTINCT columns AS alias
  FROM table
 LIMIT m;
```

## 練習題 01-05

練習題會涵蓋四個學習資料庫，記得要依據題目的需求，調整編輯器選單的學習資料庫，在自己電腦的 SQLiteStudio 寫出跟預期輸出相同的 SQL 敘述，寫作過程如果卡關了，可以參考附錄二「練習題參考解答」。

### 01. 從 `twElection2020` 資料庫的 `admin_regions` 資料表選擇所有變數，並且使用 `LIMIT 5` 顯示前五列資料，參考下列的預期查詢結果。

預期輸出：(5, 4) 的查詢結果。

+----+-----------+-----------+-----------+
| id | county    | town      | village   |
+----+-----------+-----------+-----------+
| 1  | 南投縣 | 中寮鄉 | 中寮村 |
+----+-----------+-----------+-----------+
| 2  | 南投縣 | 中寮鄉 | 內城村 |
+----+-----------+-----------+-----------+
| 3  | 南投縣 | 中寮鄉 | 八仙村 |
+----+-----------+-----------+-----------+
| 4  | 南投縣 | 中寮鄉 | 和興村 |
+----+-----------+-----------+-----------+
| 5  | 南投縣 | 中寮鄉 | 崁頂村 |
+----+-----------+-----------+-----------+
5 rows in set (0.00 sec)

### 02. 從 `nba` 資料庫的球隊資料表 `teams` 中選擇 `confName`、`divName`、`fullName` 三個變數，並且使用 `LIMIT 10` 顯示前十列資料，參考下列預期的查詢結果。

預期輸出：(10, 3) 的查詢結果。

+----------+-----------+-----------------------+
| confName | divName   | fullName              |
+----------+-----------+-----------------------+
| East     | Southeast | Atlanta Hawks         |
+----------+-----------+-----------------------+
| East     | Atlantic  | Boston Celtics        |
+----------+-----------+-----------------------+
| East     | Central   | Cleveland Cavaliers   |
+----------+-----------+-----------------------+
| West     | Southwest | New Orleans Pelicans  |
+----------+-----------+-----------------------+
| East     | Central   | Chicago Bulls         |
+----------+-----------+-----------------------+
| West     | Southwest | Dallas Mavericks      |
+----------+-----------+-----------------------+
| West     | Northwest | Denver Nuggets        |
+----------+-----------+-----------------------+
| West     | Pacific   | Golden State Warriors |
+----------+-----------+-----------------------+
| West     | Southwest | Houston Rockets       |
+----------+-----------+-----------------------+
| West     | Pacific   | LA Clippers           |
+----------+-----------+-----------------------+
10 rows in set (0.00 sec)

### 03. 從 `nba` 資料庫的球員資料表 `players` 中選擇 `firstName`、`lastName` 兩個變數，並依序取別名為 `first_name`、`last_name`，使用 `LIMIT 5` 顯示前五列資料，參考下列預期的查詢結果。

預期輸出：(5, 2) 的查詢結果。

+------------+-----------+
| first_name | last_name |
+------------+-----------+
| LeBron     | James     |
+------------+-----------+
| Carmelo    | Anthony   |
+------------+-----------+
| Udonis     | Haslem    |
+------------+-----------+
| Dwight     | Howard    |
+------------+-----------+
| Andre      | Iguodala  |
+------------+-----------+
5 rows in set (0.00 sec)

### 04. 從 `twElection2020` 資料庫的 `admin_regions` 資料表選擇「不重複」的縣市（`county`），參考下列的預期查詢結果。

預期輸出：(22, 1) 的查詢結果。

+-------------------+
| distinct_counties |
+-------------------+
| 南投縣         |
+-------------------+
| 嘉義市         |
+-------------------+
| 嘉義縣         |
+-------------------+
| 基隆市         |
+-------------------+
| 宜蘭縣         |
+-------------------+
| 屏東縣         |
+-------------------+
| 彰化縣         |
+-------------------+
| 新北市         |
+-------------------+
| 新竹市         |
+-------------------+
| 新竹縣         |
+-------------------+
| 桃園市         |
+-------------------+
| 澎湖縣         |
+-------------------+
| 臺中市         |
+-------------------+
| 臺北市         |
+-------------------+
| 臺南市         |
+-------------------+
| 臺東縣         |
+-------------------+
| 花蓮縣         |
+-------------------+
| 苗栗縣         |
+-------------------+
| 連江縣         |
+-------------------+
| 金門縣         |
+-------------------+
| 雲林縣         |
+-------------------+
| 高雄市         |
+-------------------+
22 rows in set (0.01 sec)

### 05. 從 `nba` 資料庫的 `teams` 資料表選擇「不重複」的分組（`divName`），參考下列的預期查詢結果。

預期輸出：(6, 1) 的查詢結果。

+--------------------+
| distinct_divisions |
+--------------------+
| Southeast          |
+--------------------+
| Atlantic           |
+--------------------+
| Central            |
+--------------------+
| Southwest          |
+--------------------+
| Northwest          |
+--------------------+
| Pacific            |
+--------------------+
6 rows in set (0.00 sec)